In [1]:
# Download all packages
from google.colab import files, output, drive

!rm -rf darknet
!git clone https://github.com/AlexeyAB/darknet

output.clear()
print("All packages downloaded!")

All packages downloaded!


In [2]:
# Load all packages
import os
import shutil

In [3]:
# Set up Yolov4
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make
!rm yolov4.conv.137
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

output.clear()
print("Yolov4 Ready!")

Yolov4 Ready!


In [4]:
# Connect colab to kaggle
%cd ../
!rm kaggle.json
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

output.clear()
print("Colab connected to Kaggle!")

Colab connected to Kaggle!


In [12]:
# Mount colab to google drive
drive.mount('/content/gdrive')

output.clear()
print("Colab connected to Drive!")

Colab connected to Drive!


In [6]:
# Setup datasets for Weapons
%cd darknet
!rm weapons-in-images-segmented-videos.zip
!kaggle datasets download -d jubaerad/weapons-in-images-segmented-videos
!rm -rf Weapons-in-Images
!unzip weapons-in-images-segmented-videos.zip
!rm weapons-in-images-segmented-videos.zip
!rm -rf "Weapon in Images (Segmented Video)"
!rm -r data/*
!mkdir -p data/{obj,test}

directory_weapons = "./Weapons-in-Images/Weapons-in-Images"
all_data_weapons = os.listdir(directory_weapons)
for data in all_data_weapons:
    real_data, extension_data = data.split(".")
    if extension_data == "jpg" and f"{real_data}.txt" not in all_data_weapons:
        os.remove(f"./Weapons-in-Images/Weapons-in-Images/{data}")

all_data_weapons = os.listdir(directory_weapons)
all_data_weapons.sort()
train_composition = int(((len(all_data_weapons) / 2) * 0.8) * 2)
test_composition = len(all_data_weapons) - train_composition
count = 0
for data in all_data_weapons:
    if count < test_composition:
        shutil.move(os.path.join(directory_weapons, data), "data/test/")
    elif count < train_composition:
        shutil.move(os.path.join(directory_weapons, data), "data/obj/")
    count += 1

!rm -rf Weapons-in-Images
output.clear()
print("Datasets ready!")

Datasets ready!


In [7]:
# Create train.txt
image_files = []
os.chdir(os.path.join("data", "obj"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/obj/" + filename)
os.chdir("..")
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

output.clear()
print("train.txt created!")

train.txt created!


In [8]:
# Create test.txt
image_files = []
os.chdir(os.path.join("data", "test"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/test/" + filename)
os.chdir("..")
with open("test.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

output.clear()
print("test.txt created!")

test.txt created!


In [9]:
# Set yolov4 config and backup dataset
!>data/obj.data
!>data/obj.names

output.clear()
print("Yolov4 configured and backup dataset!")
"""
- MAKE SURE SET CONFIG FOR CUSTOM yolov4-custom.cfg
- MAKE SURE SET YOUR DATASET obj.data AND obj.names
"""
assert False

Yolov4 configured and backup dataset!


AssertionError: ignored

In [14]:
# Train model
!./darknet detector train \
data/obj.data \
cfg/yolov4-custom.cfg \
./yolov4.conv.137 \
-dont_show

output.clear()
print("Train done!")

Train done!


In [17]:
# Checking Mean Average Precision Each Weights
print("============================")
print("LAST WEIGHTS")
!./darknet detector map \
data/obj.data \
cfg/yolov4-custom.cfg \
/content/gdrive/MyDrive/"Week 3"/Task/google_colab_data/backup/yolov4-custom_last.weights

print("============================")
print("1000 MAX BATCHERS")
!./darknet detector map \
data/obj.data \
cfg/yolov4-custom.cfg \
/content/gdrive/MyDrive/"Week 3"/Task/google_colab_data/backup/yolov4-custom_1000.weights



LAST WEIGHTS
 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 